In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import glob
import os

In [2]:
df_list = []
os_list = os.listdir()
os_list.sort()

csv_list = [file for file in os_list if file.endswith('.csv')]
csv_list.pop(0)
json_list = [file for file in os_list if file.endswith('.json')]

for i,val in enumerate(csv_list):
    
    # open json and create dict from file
    with open(json_list[i]) as json_data:
        data = json.load(json_data)
    
    df_json = pd.DataFrame(data['selectedAttributes'])
    
    lga_name_to_title = dict(zip(df_json['name'],df_json['title']))
    
    # open csv file and clean
    df = pd.read_csv(csv_list[i], index_col=None, header=0)
    
    clean_cols = []

    for col in df.columns:
        clean_cols.append(lga_name_to_title[col.strip(' ')].split(' Age')[0].replace('Persons ','').replace('Females ','').replace('Males ','').replace('emales ','').replace('ales ',''))
    
    df.columns = clean_cols
    df = df.astype({"LGA Code 2016": str}) 
    #
    
    
    
    #
    df = df.groupby(df.columns, axis=1).sum()
    
    col_list = df.columns

    for col in col_list:
        if (('Total' in col) and ('LGA' not in col)):
            df=df.drop(col,axis=1)
            
    df_list.append(df)

In [3]:
df_list[0][df_list[0]['LGA Name 2016'] == 'Latrobe (M) (Tas.)']

,Afghanistan,Australia,Bangladesh,Bosnia and Herzegovina,Cambodia,Canada,Chile,China excludes SARs and Taiwan,Croatia,Egypt,...,France,Germany,Greece,Hong Kong SAR of China,India,Indonesia,Iran,Iraq,LGA Code 2016,LGA Name 2016
531,0,4418,0,0,0,0,0,0,0,0,...,0,6,0,0,0,0,0,7,63810,Latrobe (M) (Tas.)


In [4]:
df = pd.concat(df_list,axis=1)

In [5]:

df = df.groupby(df.columns, axis=1).sum()

In [6]:
df = df.rename(columns = {'China excludes SARs and Taiwan':'China', 
                         'Korea Republic of South':'South Korea',
                         'South Eastern Europe nfd':'South Eastern Europe',
                         'The Former Yugoslav Republic of Macedonia':'North Macedonia',
                         'Hong Kong SAR of China':'Hong Kong',
                         'LGA Name 2016':'LGA Name'})

In [7]:
df = df.drop(columns = ['Born elsewhere','Country of birth not stated','LGA Code 2016'])

In [8]:
second_col = df.pop('LGA Name')

df.insert(0, 'LGA Name', second_col)

In [9]:
for i,name in enumerate(df['LGA Name']):
    df['LGA Name'][i] = df['LGA Name'][i].split(')')[0]+')'


<ipython-input-9-bf6dee3ffc8c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LGA Name'][i] = df['LGA Name'][i].split(')')[0]+')'


In [10]:
df

,LGA Name,Afghanistan,Australia,Bangladesh,Bosnia and Herzegovina,Cambodia,Canada,Chile,China,Croatia,...,South Eastern Europe,Sri Lanka,Taiwan,Thailand,North Macedonia,Turkey,United States of America,Vietnam,Wales,Zimbabwe
0,Albury (C),34,83170,170,0,10,80,17,238,89,...,62,200,3,125,0,0,191,65,78,58
1,Armidale Regional (A),0,46805,108,0,0,99,15,558,0,...,0,120,80,43,0,0,310,93,67,45
2,Ballina (A),0,67975,3,0,12,163,58,108,19,...,8,41,9,124,0,9,282,43,101,58
3,Balranald (A),0,3744,0,0,0,0,0,0,0,...,0,0,8,12,0,0,0,0,0,0
4,Bathurst Regional (A),36,69062,65,3,12,108,21,240,43,...,28,106,0,93,5,0,120,114,43,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,Darwin (C),148,89904,567,47,124,363,47,1902,31,...,16,272,144,167,3,13,163,481,77,221
541,East Arnhem (R),0,17429,0,0,0,0,0,0,0,...,0,0,0,0,0,0,8,0,0,0
542,Katherine (T),0,13136,0,0,3,11,0,54,3,...,0,31,7,17,0,0,51,40,11,35
543,Litchfield (M),14,33715,0,0,124,46,8,59,9,...,12,14,14,113,4,0,70,341,31,115


In [11]:
df1 = pd.read_csv('lga_g09a_country_of_birth_by_age_by_sex_census_2016-8577272101096579498.csv')

In [12]:
df.insert(1, 'LGA Code', df1[' lga_code_2016'])

In [15]:
df['LGA Name'] = df['LGA Name'].apply(lambda x: x.split(' (')[0])

In [22]:
clean_cols = []

for col in df.columns:
    clean_cols.append(col.replace(' ','_').lower())

df.columns = clean_cols

In [23]:
df

,lga_name,lga_code,afghanistan,australia,bangladesh,bosnia_and_herzegovina,cambodia,canada,chile,china,...,south_eastern_europe,sri_lanka,taiwan,thailand,north_macedonia,turkey,united_states_of_america,vietnam,wales,zimbabwe
0,Albury,10050,34,83170,170,0,10,80,17,238,...,62,200,3,125,0,0,191,65,78,58
1,Armidale Regional,10130,0,46805,108,0,0,99,15,558,...,0,120,80,43,0,0,310,93,67,45
2,Ballina,10250,0,67975,3,0,12,163,58,108,...,8,41,9,124,0,9,282,43,101,58
3,Balranald,10300,0,3744,0,0,0,0,0,0,...,0,0,8,12,0,0,0,0,0,0
4,Bathurst Regional,10470,36,69062,65,3,12,108,21,240,...,28,106,0,93,5,0,120,114,43,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,Darwin,71000,148,89904,567,47,124,363,47,1902,...,16,272,144,167,3,13,163,481,77,221
541,East Arnhem,71300,0,17429,0,0,0,0,0,0,...,0,0,0,0,0,0,8,0,0,0
542,Katherine,72200,0,13136,0,0,3,11,0,54,...,0,31,7,17,0,0,51,40,11,35
543,Litchfield,72300,14,33715,0,0,124,46,8,59,...,12,14,14,113,4,0,70,341,31,115


In [24]:
df.to_csv('clean_country_of_birth_by_lga.csv',index=False)